In [1]:
import pdal
import numpy as np

In [2]:
%load_ext autotime

In [3]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 1
    }
  ]
}"""

In [4]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
pts = r.arrays[0]

In [5]:
pts

array([(333499.29, 7395564.35, 735.34, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36606547),
       (333499.23, 7395565.79, 735.41, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36609045),
       (333499.2 , 7395566.79, 735.25, 15, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36610547),
       ...,
       (333520.96, 7395571.  , 732.5 ,  8, 1, 1, 1, 0, 6,  13., 40, 6, 363044.34273991),
       (333521.09, 7395571.84, 731.83, 18, 1, 1, 1, 0, 6,  13., 40, 6, 363044.3609089 ),
       (333520.95, 7395571.91, 729.86, 18, 1, 1, 0, 0, 6,  13., 40, 6, 363044.36412489)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8')])

In [6]:
pt_edif = pts['Classification'] == 6
len(pts[pt_edif])

300792

In [7]:
pt_ground = pts['Classification'] == 2
len(pts[pt_ground])

15890

In [8]:
len(pts)

471962

In [9]:
len(pts[pt_ground | pt_edif])

316682

In [10]:
# Escolhendo um ponto aleatório no Ground

pt = np.random.choice(pts[pt_ground])

In [11]:
coord = ['X', 'Y', 'Z']

In [12]:
pt[coord]

(333340.48, 7395505.53, 730.03)

In [13]:
# Centralizando a visão em 1.5 metros acima do solo do ponto escolhido

pts_0 = np.array([pts[pt_ground | pt_edif]['X'] - pt['X'],
         pts[pt_ground | pt_edif]['Y'] - pt['Y'],
         pts[pt_ground | pt_edif]['Z'] - (pt['Z'] + 1.5)]).T

In [14]:
## TODO
# Reprojetar os pontos para a circunferência da terra

In [15]:
## Filtrando pontos acima da visão horizontal

pts_0 = pts_0[pts_0[:, 2] >= 0.0]

In [16]:
len(pts_0)

295911

In [17]:
# Calcluando Azimute, ou melhor, um angulo começando Leste 
# crescendo no sentido anti-horario e negativo para todo Y < 0

teste_az = [[0., 1],
           [1., 0.],
           [0., -1.],
           [-1., 0.]]

teste_az = np.array(teste_az)

az = np.arctan2(teste_az[:, 1], teste_az[:, 0]) * 180 / np.pi
az

array([ 90.,   0., -90., 180.])

In [21]:
# Calculando o angulo de orientação de cada ponto

orient = np.arctan2(pts_0[:, 1], pts_0[:, 0]) * 180 / np.pi
# len(orient)

In [22]:
# Calculando o angulo de altura de cada ponto

# np.arctan2(10, 1000) * 180 / np.pi
# np.sqrt(np.square(4) + np.square(3))

inclin = np.arctan2(pts_0[:, 2], np.sqrt(np.square(pts_0[:, 0]) + np.square(pts_0[:, 1]))) * 180 / np.pi

In [23]:
# Calculando a distância até o ponto

np.sqrt(np.square(1) + np.square(1) + np.square(10))

10.099504938362077

In [24]:
# sorted(inclin, reverse=True)

In [25]:
orient

array([20.32360685, 20.78628867, 21.10472835, ..., 17.1965555 ,
       19.93848683, 20.16044186])

In [26]:
# Separando em fases

qt_fases = 60
fases = np.arange(-180., 180., 360./qt_fases)

pt_bins = np.digitize(orient, fases)

In [27]:
np.histogram(orient, fases)

(array([ 4846,  2218,  1689,  3287,  3529,  2806,  5649,  6983, 12856,
        20260, 24715, 26459, 22437, 21586, 23244, 13736,  7653,  7406,
         6132,  6957,  7650,  4120,  3405,  3106,  3875,  3315,  3597,
         2928,  4300,  2549,  2797,  3847,  2618,  1304,  1184,   803,
          564,   362,   340,   456,   378,   433,   559,   495,   496,
          480,   447,   280,   162,   241,   374,   418,   461,   594,
          798,  1286,  1089,  3047,  1842]),
 array([-180., -174., -168., -162., -156., -150., -144., -138., -132.,
        -126., -120., -114., -108., -102.,  -96.,  -90.,  -84.,  -78.,
         -72.,  -66.,  -60.,  -54.,  -48.,  -42.,  -36.,  -30.,  -24.,
         -18.,  -12.,   -6.,    0.,    6.,   12.,   18.,   24.,   30.,
          36.,   42.,   48.,   54.,   60.,   66.,   72.,   78.,   84.,
          90.,   96.,  102.,  108.,  114.,  120.,  126.,  132.,  138.,
         144.,  150.,  156.,  162.,  168.,  174.]))

In [28]:
np.unique(pt_bins)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60])

In [29]:
np.arange(1, qt_fases + 1)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60])

In [30]:
list(map(lambda x: [x, np.nanmax(inclin[pt_bins == x])] if len(inclin[pt_bins == x]) > 0 else [x, 0.], np.arange(1, qt_fases + 1)))
# list(map(lambda x: len(orient[pt_bins == x]), fases))
# np.nanmax(inclin[pt_bins == 3]) if len(inclin[pt_bins == 3]) > 0

[[1, 85.91739960637011],
 [2, 85.05915229442553],
 [3, 84.52537566717291],
 [4, 85.33186915597976],
 [5, 85.51559985524625],
 [6, 83.04968789000661],
 [7, 87.8564090357346],
 [8, 88.17239229742398],
 [9, 77.67213585666134],
 [10, 88.09681517678403],
 [11, 76.89440942142657],
 [12, 84.99299446930976],
 [13, 87.30860816567726],
 [14, 85.31419603160214],
 [15, 86.90817485389135],
 [16, 86.1356492145282],
 [17, 81.57397906825868],
 [18, 86.73438157819683],
 [19, 84.46275940780555],
 [20, 88.05554478926817],
 [21, 81.11864936952624],
 [22, 83.8110749206603],
 [23, 85.97071640972874],
 [24, 86.10574476479249],
 [25, 86.24398978021787],
 [26, 83.06157895565384],
 [27, 81.02520861089025],
 [28, 68.39057862785269],
 [29, 22.69477472380728],
 [30, 21.692195247900756],
 [31, 23.154874232007113],
 [32, 24.38189574291941],
 [33, 29.723144118527358],
 [34, 26.4134524209064],
 [35, 31.133313182160038],
 [36, 28.79369536649014],
 [37, 28.072675713570145],
 [38, 26.970295372336196],
 [39, 23.2707521210

In [27]:
# A relação entre a densidade de pontos e a distância para obstrução do céu visível
# Quanto mais próximo um ponto está do observador, maior a obstrução que ele deve promover
# Os parâmetros portanto são distância ao observador e a distância máxima entre pontos

# Os bins então vão responder não mais a um angulo, mas uma faixa de angulos

# o Angulo é dado por arctan2(dist_max_entre_pontos/2, distancia_do_observador)

# densidade em raio
raio = 1

desvio_graus = np.degrees(np.arctan2(raio/2, \
                      np.sqrt(np.square(pts_0[:, 0]) + \
                              np.square(pts_0[:, 1]) + \
                              np.square(pts_0[:, 2]))) * 2)

desvio_graus

# Depois refatorar a função de máximo ângulo da fase

array([0.19749032, 0.19660078, 0.19598865, ..., 0.18791985, 0.18742728,
       0.18744317])

In [28]:
# Refatorando o calculo das fases

# list(map(lambda x: \
#          [x, np.nanmax(inclin[pt_bins == x])] if len(inclin[pt_bins == x]) > 0 else [x, 0.], np.arange(1, qt_fases + 1)))

orient + desvio_graus
list(map(lambda x: x, fases))

len(pts_0[((orient + desvio_graus >= 6.) | (orient - desvio_graus >= 6.) ) & ((orient + desvio_graus < 6. + 6) | (orient - desvio_graus < 6. + 6) ) ])

2677

In [ ]:
# Calcular o percentual de cada fuso
# Sendo que a área do fuso é a área da esfera 4 x Pi x r x r dividido pela quantidade de fusos
# em cada fuso subtrair a quantidade de céu visível, dao pela fórmula da calota da esfera
# 2 x Pi x r x h (sendo necessário calcular h e considenrando r constante)

In [ ]:
## TODO
# Pensar em como iterar sobre as folhas!

In [38]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.6f}".format(x)})

In [39]:
# Verificando a resolução mínima
# O tamanho em ângulo do fuso e a distância ao ponto do observador são parâmetros

angulo_fase = 360. / qt_fases
angulo_fase

distancia = np.array([1, 10, 100, 500, 1000, 10000])

np.sin(np.deg2rad(angulo_fase)) * distancia

array([0.105, 1.045, 10.453, 52.264, 104.528, 1045.285])